In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor

In [2]:
# building_id = 32
# subset_df = pd.read_csv('/Users/veronica/Library/CloudStorage/GoogleDrive-veronicahangsan@gmail.com/.shortcut-targets-by-id/1FsOPywSgK_wZmrVrSTBVi4q8G3Mg_yMJ/Team-Fermata-Energy/processed_data/subset20.csv')
# processed_df = pd.read_csv(f'/Users/veronica/Library/CloudStorage/GoogleDrive-veronicahangsan@gmail.com/.shortcut-targets-by-id/1FsOPywSgK_wZmrVrSTBVi4q8G3Mg_yMJ/Team-Fermata-Energy/processed_data/processed_weather_load_w_timestamp/{building_id}.csv')


md_path = '/Users/veronica/Library/CloudStorage/GoogleDrive-veronicahangsan@gmail.com/.shortcut-targets-by-id/1FsOPywSgK_wZmrVrSTBVi4q8G3Mg_yMJ/Team-Fermata-Energy/processed_data/md_one_hot_encoded_subset20.csv'
PATH = '/Users/veronica/Library/CloudStorage/GoogleDrive-veronicahangsan@gmail.com/.shortcut-targets-by-id/1FsOPywSgK_wZmrVrSTBVi4q8G3Mg_yMJ/Team-Fermata-Energy/processed_data/processed_weather_load_w_timestamp/'
json_file = '../data/subset20_20_data.json'

In [3]:
md = pd.read_csv(md_path)
with open(json_file, 'r') as file:
    json_data = json.load(file)

In [4]:
md.head

<bound method NDFrame.head of       bldg_id  in.state  in.vintage   in.sqft  \
0      105885        10           3  750000.0   
1      305819        40           2  150000.0   
2      305934        40           4  350000.0   
3      317044        40           3  350000.0   
4          32         1           6   37500.0   
...       ...       ...         ...       ...   
6396   331879        42           1   37500.0   
6397   331938        42           3   37500.0   
6398   332036        42           3    3000.0   
6399   332037        42           1   17500.0   
6400   332051        42           2   37500.0   

      in.building_america_climate_zone_Cold  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   
...                                     ...   
6396                                      0   
6397 

In [5]:
md.columns

Index(['bldg_id', 'in.state', 'in.vintage', 'in.sqft',
       'in.building_america_climate_zone_Cold',
       'in.building_america_climate_zone_Hot-Dry',
       'in.building_america_climate_zone_Hot-Humid',
       'in.building_america_climate_zone_Marine',
       'in.building_america_climate_zone_Mixed-Dry',
       'in.building_america_climate_zone_Mixed-Humid',
       'in.building_america_climate_zone_Subarctic',
       'in.building_america_climate_zone_Very Cold', 'in.iso_rto_region_CAISO',
       'in.iso_rto_region_ERCOT', 'in.iso_rto_region_MISO',
       'in.iso_rto_region_NEISO', 'in.iso_rto_region_NYISO',
       'in.iso_rto_region_PJM', 'in.iso_rto_region_SPP',
       'in.iso_rto_region_none', 'in.heating_fuel_DistrictHeating',
       'in.heating_fuel_Electricity', 'in.heating_fuel_FuelOil',
       'in.heating_fuel_NaturalGas', 'in.heating_fuel_Propane',
       'in.interior_lighting_generation_gen1_t12_incandescent',
       'in.interior_lighting_generation_gen2_t8_halogen',
     

In [6]:
train_bldg_ids = json_data.get("train_bldg_ids", [])
test_bldg_ids = json_data.get("test_bldg_ids", [])

In [7]:
train_bldg_ids = [filename.split('.')[0] for filename in train_bldg_ids]
test_bldg_ids = [filename.split('.')[0] for filename in test_bldg_ids]

md['bldg_id'] = md['bldg_id'].astype(str).str.strip()

In [8]:
education_buildings = md[md['in.comstock_building_type_group_Education'] == 1]
education_bldg_ids = education_buildings['bldg_id'].unique()

In [9]:
print(f"Education Building IDs: {education_bldg_ids[:10]}")

Education Building IDs: ['1025' '3050' '3587' '5623' '5778' '6141' '7137' '10351' '12020' '13244']


In [10]:
valid_train_bldgs = set(train_bldg_ids).intersection(education_bldg_ids)
valid_test_bldgs = set(test_bldg_ids).intersection(education_bldg_ids)

print(f"Valid train buildings: {valid_train_bldgs}")
print(f"Valid test buildings: {valid_test_bldgs}")

Valid train buildings: {'40784', '142128', '195927', '104791', '200025', '260641', '185619', '87513', '229930', '115516', '216573', '157920', '213035', '140525', '317195', '100509', '222812', '220209', '76284', '325574', '160500', '49168', '68162', '325674', '51767', '32211', '132028', '286199', '257381', '109381', '5623', '166330', '241943', '324210', '175636', '300292', '286832', '174061', '46736', '13244', '143155', '85280', '330851', '191781', '320417', '131132', '130424', '147211', '329909', '141937', '91960', '142950', '236343', '138433', '184684', '114047', '105735', '202677', '175521', '76225', '15554', '38370', '177619', '132936', '116162', '297199', '242077', '122765', '196938', '112004', '130137', '146220', '193183', '197094', '317886', '290293', '172879', '325656', '129232', '282426', '39806', '324067', '28870', '233818', '100131', '309670', '18768', '306876', '10351', '97705', '302537', '173742', '33545', '129318', '198468', '256321', '157959', '268181', '200843', '224597'

In [11]:
print(f"Number of valid training buildings: {len(valid_train_bldgs)}")
print(f"Number of valid testing buildings: {len(valid_test_bldgs)}")


Number of valid training buildings: 120
Number of valid testing buildings: 33


In [12]:
def load_data_for_building(filename, directory):
    try:
        file_path = f"{directory}/{filename}.csv"
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error loading file {filename}: {e}")
        return None


In [13]:
def prepare_data_for_building(df_load, md):
    df_load['bldg_id'] = df_load['bldg_id'].astype(str)
    md['bldg_id'] = md['bldg_id'].astype(str)

    for i in range(1, 97):
        df_load[f"shift_{i}"] = df_load["out.electricity.total.energy_consumption"].shift(i)

    if 'timestamp' in df_load.columns:
        df_load['timestamp'] = pd.to_datetime(df_load['timestamp'])
        
        datetime_columns = {
            'hour': df_load['timestamp'].dt.hour,
            'day_of_week': df_load['timestamp'].dt.dayofweek,
            'day_of_year': df_load['timestamp'].dt.dayofyear,
            'month': df_load['timestamp'].dt.month,
            'year': df_load['timestamp'].dt.year
        }
        
        df_load = pd.concat([df_load, pd.DataFrame(datetime_columns)], axis=1)
        df_load = df_load.drop(columns=['timestamp'])  # drop timestamp

    merged_df = df_load.merge(md, on='bldg_id', how='left')
    merged_df = merged_df.drop(['bldg_id'], axis=1)

    # drop missing values
    merged_df = merged_df.dropna()
    
    return merged_df

In [14]:
def smape(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    denominator = np.abs(actual) + np.abs(predicted)
    diff = np.abs(actual - predicted) / denominator
    diff[denominator == 0] = 0.0
    return 200 * np.mean(diff)

In [15]:
def plot_results(y_test, y_pred):
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.index, y_pred.values, label='Predicted', color='red', zorder=2)
    plt.plot(y_test.index, y_test.values, label='Actual', color='blue', zorder=1)
    plt.xlabel('Time')
    plt.ylabel('Electricity Consumption')
    plt.title('Random Forest: Actual vs Predicted Electricity Consumption')
    plt.legend()
    plt.show()

In [16]:
def train_random_forest_model(directory, valid_train_bldgs, valid_test_bldgs, target_column='out.electricity.total.energy_consumption'):
    smape_train_list = []
    smape_test_list = []

    model = RandomForestRegressor(n_estimators=20, random_state=42, n_jobs=-1)

    for building_id in valid_train_bldgs:
        train_df = load_data_for_building(building_id, directory)
        if train_df is not None and target_column in train_df:
            train_df_prepared = prepare_data_for_building(train_df, md)
            
            if train_df_prepared is not None and len(train_df_prepared) > 0:
                
                # features (X) and target (y)
                X_train = train_df_prepared.drop(columns=[target_column])
                y_train = train_df_prepared[target_column]
                
                # Random Forest model
                model.fit(X_train, y_train)
                
                y_train_pred = model.predict(X_train)
                
                smape_train = smape(y_train, y_train_pred)
                smape_train_list.append(smape_train)
            
            else:
                print(f"Prepared data is empty for {building_id}")
        else:
            print(f"Skipping {building_id} due to missing data")

    # --- testing ---
    for building_id in valid_test_bldgs:   
        test_df = load_data_for_building(building_id, directory)
        if test_df is not None and target_column in test_df:
            test_df_prepared = prepare_data_for_building(test_df, md)
            
            if test_df_prepared is not None and len(test_df_prepared) > 0:
                
                # features (X) and target (y)
                X_test = test_df_prepared.drop(columns=[target_column])
                y_test = test_df_prepared[target_column]
                
                y_test_pred = model.predict(X_test)
                
                smape_test = smape(y_test, y_test_pred)
                smape_test_list.append(smape_test)
            
            else:
                print(f"Prepared data is empty for {building_id}")
        else:
            print(f"Skipping {building_id} due to missing data")

    # avg SMAPE for both training and testing sets
    avg_smape_train = np.mean(smape_train_list)
    avg_smape_test = np.mean(smape_test_list)

    print(f"Average SMAPE (training set): {avg_smape_train:.4f}")
    print(f"Average SMAPE (testing set): {avg_smape_test:.4f}")
    
    # overall (combined) average SMAPE
    overall_avg_smape = (avg_smape_train + avg_smape_test) / 2
    return overall_avg_smape


In [17]:
overall_smape = train_random_forest_model(PATH, valid_train_bldgs, valid_test_bldgs)

print(f"Final overall SMAPE: {overall_smape}")

Average SMAPE (training set): 1.1749
Average SMAPE (testing set): 53.7525
Final overall SMAPE: 27.46367750696517
